In [2]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pickle
import numpy as np

# Define the input data schema
class LoanApplication(BaseModel):
    person_age: int
    person_income: float
    person_home_ownership: str
    person_emp_length: float
    loan_intent: str
    loan_grade: str
    loan_amnt: int
    loan_int_rate: float
    loan_percent_income: float
    cb_person_default_on_file: str
    cb_person_cred_hist_length: int


# Mapping for categorical variables
person_home_ownership_map = {"RENT": 2, "OWN": 3, "MORTGAGE": 0, "OTHER": 1}
loan_grade_map = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6}
loan_intent_map = {
    "DEBT CONSOLIDATION": 0,
    "HOME IMPROVEMENT": 1,
    "VENTURE": 5,
    "PERSONAL": 4,
    "MEDICAL": 3,
    "EDUCATION":1
}
default_on_file_map = {"Yes": 1, "No": 0}




In [3]:
# Load the trained ML model
def load_light_model():
    try:
        with open("reserch\lightgbm_model.pkl", "rb") as f:
            return pickle.load(f)
    except Exception as e:
        raise RuntimeError(f"Error loading model: {e}")

def load_cat_model():
    try:
        with open("CB_classifier.pkl",'rb') as f:
            return pickle.load(f)
    except Exception as e:
        raise RuntimeError(f"Error loading model : {e}")

In [4]:
# Make prediction
def make_prediction(model, input_data):
    try:
        prediction = model.predict(input_data)
        return prediction
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error making prediction: {e}")
def make_proba(model,input_data):
    try:
        prediction = model.predict_proba(input_data)
        return prediction
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error making prediction: {e}")


In [5]:
data_dict = {
  "person_age": 25,
  "person_income": 10000,
  "person_home_ownership": "Own",
  "person_emp_length": 12,
  "loan_intent": "HOME IMPROVEMENT",
  "loan_grade": "D",
  "loan_amnt": 1500,
  "loan_int_rate": 5.6,
  "loan_percent_income": 0.067,
  "cb_person_default_on_file": "No",
  "cb_person_cred_hist_length": 12
}

data_dict_2 = {
      "person_age": 25,
  "person_income": 25000,
  "person_home_ownership": "RENT",
  "person_emp_length": 3.0,
  "loan_intent": "VENTURE",
  "loan_grade": "C",
  "loan_amnt": 3500,
  "loan_int_rate": 13.49,
  "loan_percent_income": 0.14,
  "cb_person_default_on_file": "Yes",
  "cb_person_cred_hist_length": 3
}
# 25,25,25000,RENT,3.0,VENTURE,C,3500,13.49,0.14,Y,3,1

In [6]:
def preprocess_data_1(data: dict):
    input_data = np.array([
        data["person_age"],
        data["person_income"],
        person_home_ownership_map[data["person_home_ownership"]],
        data["person_emp_length"],
        loan_grade_map[data["loan_grade"]],
        loan_intent_map[data["loan_intent"]],
        data["loan_amnt"],
        data["loan_int_rate"],
        data["loan_percent_income"],
        default_on_file_map[data["cb_person_default_on_file"]],
        data["cb_person_cred_hist_length"]
    ]).reshape(1, -1)
    return input_data

In [7]:
preprocess_data_1(data_dict_2)

array([[2.500e+01, 2.500e+04, 2.000e+00, 3.000e+00, 2.000e+00, 5.000e+00,
        3.500e+03, 1.349e+01, 1.400e-01, 1.000e+00, 3.000e+00]])

In [8]:
model = load_cat_model()
input_data = preprocess_data_1(data_dict_2)
print(input_data)


[[2.500e+01 2.500e+04 2.000e+00 3.000e+00 2.000e+00 5.000e+00 3.500e+03
  1.349e+01 1.400e-01 1.000e+00 3.000e+00]]


In [19]:
# input_dataa = np.array([22,30000,2,3.0,3,3,5000,16.49,0.17,0,4,1]).reshape(1,-1)
input_dataa = np.array([22,30000,"RENT",3.0,"MEDICAL","D",5000,16.49,0.17,"N",4]).reshape(1,-1)
print(input_data)
output = make_prediction(model,input_dataa)
op = make_proba(model,input_dataa)
print(op)
print(output)

[[2.500e+01 2.500e+04 2.000e+00 3.000e+00 2.000e+00 5.000e+00 3.500e+03
  1.349e+01 1.400e-01 1.000e+00 3.000e+00]]
[[0.27242508 0.72757492]]
[1]


got it yahi karna hai bas done uper we got zero

In [10]:
input_dataa = np.array([27,100000,'RENT',1.0,'HOMEIMPROVEMENT','C',5000,13.57,0.05,'Y',7]).reshape(1,-1)
output = make_prediction(model,input_dataa)
op = make_proba(model,input_dataa)
print(op)
print(output)

[[0.98037668 0.01962332]]
[0]


In [18]:
data_dict_3 = {"person_age":25,"person_income":25000,"person_home_ownership":"RENT","person_emp_length":3,"loan_intent":"VENTURE","loan_grade":"C","loan_amnt":3500,"loan_int_rate":13.49,"loan_percent_income":0.071,"cb_person_default_on_file":"Yes","cb_person_cred_hist_length":3}

preprocess_data_1(data_dict_3)
output = make_prediction(model,input_dataa)
op = make_proba(model,input_dataa)
print(op)
print(output)

[[0.98037668 0.01962332]]
[0]


In [ ]:
# Initialize FastAPI app
app = FastAPI()
model = load_cat_model()

@app.post("/predict")
async def predict_loan_application(data):
    input_data = preprocess_data_1(data)
    output = make_prediction(model, input_data)
    return {"classification_result": output}